In [1]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import grammar as lt
from collections import Counter
from tqdm import tqdm
from joblib import Parallel, delayed
from spacy import load
import re
import yaml
import random
from pymongo import MongoClient, UpdateOne
from plotqa import match_and_generate
nlp = load('en_core_web_sm')

2023-05-10 16:59:18.691085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 16:59:19.263217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 16:59:20.041752: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-10 16:59:20.042238: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [2]:
client = MongoClient()
db = client.plotqa
G = [d for d in db.val_captions.find({"matches.0": {"$exists": True}})]
len(G)

2146603

In [5]:
ctr = lt.err_counter(G)

[('It seems like there are too many consecutive spaces here.', 1377383), ('It appears that a white space is missing.', 596158), ('A determiner may be missing.', 307959), ('Possible spelling mistake found.', 228116), ('Don’t put a space before the full stop.', 148059)]


In [7]:
ws = lt.filter_lt_messages(G, message="It seems like there are too many consecutive spaces here.")

Signature: re.sub(pattern, repl, string, count=0, flags=0)
Docstring:
Return the string obtained by replacing the leftmost
non-overlapping occurrences of the pattern in string by the
replacement repl.  repl can be either a string or a callable;
if a string, backslash escapes in it are processed.  If it is
a callable, it's passed the Match object and must return
a replacement string to be used.
File:      ~/conda/lib/python3.9/re.py
Type:      function

In [20]:
for doc in tqdm(ws):
    x = doc['caption']
    x = re.sub(r'\s+', ' ', doc['caption'])
    x = re.sub(r'(\S)\(', r'\1 (', x)
    doc['caption'] = x

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1373331/1373331 [00:11<00:00, 116305.31it/s]


In [21]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in ws)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-05-10 17:09:39.725137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 17:09:39.765615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 17:09:39.768645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 17:09:39.776737: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 17:09:39.796502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 17:09:39.827064: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 17:09:39.842070: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 17:09:39.855481: W tensorflow/compiler/tf2te

In [25]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r.get('ignore', []), 'caption': r['caption']}}) for r in ws]
wres = db.val_captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 1373331,
 'nModified': 1373331,
 'nRemoved': 0,
 'upserted': []}

In [26]:
ctr.most_common(5)

[('It seems like there are too many consecutive spaces here.', 1377383),
 ('It appears that a white space is missing.', 596158),
 ('A determiner may be missing.', 307959),
 ('Possible spelling mistake found.', 228116),
 ('Don’t put a space before the full stop.', 148059)]

In [27]:
determiner = lt.filter_lt_messages(G, message='A determiner may be missing.')

In [29]:
geo_deter = []
for d in tqdm(determiner):
    IDS = [m['rule']['id'] for m in d['matches']]
    if 'DETERMINER_GEOGRAPHICAL_WORD' in IDS:
        geo_deter.append(d)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 297709/297709 [00:00<00:00, 800288.95it/s]


In [30]:
len(determiner)

297709

In [31]:
len(geo_deter)

279365

In [36]:
lx = [c for c in lt.get_repl_values(geo_deter) if 'the' in c.lower()]

In [39]:
with open('/tmp/geodeter.json', 'w') as fout:
    json.dump([{'_id': k['_id'], 'caption': k['caption']} for k in geo_deter], fout, indent=2)

In [40]:
with open('/tmp/geodeter.json', 'r') as fin:
    fix_geo = json.load(fin)

In [42]:
fix_geo[0]

{'_id': 308,
 'caption': 'Across years, the lowest percentage of females employed in Private sector in the Maldives is 39.0.'}

In [43]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in fix_geo)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-05-10 18:43:19.105818: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 18:43:20.201274: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    5.5s
2023-05-10 18:43:22.682382: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 18:43:22.685846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 18:43:22.836633: W ten

In [44]:
x = lt.err_counter(res)

[('Possible spelling mistake found.', 53237), ('It appears that a white space is missing.', 31583), ('Don’t put a space before the full stop.', 11985), ('Possible typo: you repeated a whitespace', 6407), ('Use “a thousand”, or use a number before ‘thousand’.', 3881)]


In [46]:
updates = [UpdateOne({'_id': r['_id']}, {'$set': {'ignore': r.get('ignore', []), 'caption': r['caption']}}) for r in fix_geo]
wres = db.val_captions.bulk_write(updates)
wres.bulk_api_result

{'writeErrors': [],
 'writeConcernErrors': [],
 'nInserted': 0,
 'nUpserted': 0,
 'nMatched': 279365,
 'nModified': 278787,
 'nRemoved': 0,
 'upserted': []}

In [47]:
typos = lt.filter_lt_messages(G, message='Possible spelling mistake found.')
typos[0]

{'_id': 26,
 'question_string': 'What is the ppp conversion factor for gdp in 2002 ?',
 'template': 'data_retrieval',
 'answer': 0.90497,
 'type': 'dot',
 'template_id': 2,
 'variables': {'yvalue': 'ppp conversion factor for gdp',
  'preposition': 'in',
  'xvalue': '2002'},
 'varmatchsize': 3,
 'caption': 'The ppp conversion factor for gdp in 2002 is 0.90497.',
 'check': True,
 'matches': [{'message': 'Possible spelling mistake found.',
   'shortMessage': 'Spelling mistake',
   'replacements': [{'value': 'PPP'},
    {'value': 'pop'},
    {'value': 'PDP'},
    {'value': 'PSP'},
    {'value': 'ppm'},
    {'value': 'EPP'},
    {'value': 'MPP'},
    {'value': 'PCP'},
    {'value': 'PNP'},
    {'value': 'PPD'},
    {'value': 'PPI'},
    {'value': 'pep'},
    {'value': 'IPP'},
    {'value': 'PFP'},
    {'value': 'PKP'},
    {'value': 'pip'},
    {'value': 'ppb'},
    {'value': 'pup'},
    {'value': 'LPP'},
    {'value': 'pap'},
    {'value': 'app'},
    {'value': 'opp'},
    {'value': 'ppr'}

In [48]:
with open('/tmp/abbrev.json', 'w') as fout:
    json.dump([{'_id': k['_id'], 'caption': k['caption']} for k in typos], fout, indent=2)

In [58]:
with open('/tmp/abbrev.json', 'r') as fin:
    fix_abbrev = json.load(fin)

In [50]:
fix_abbrev[0]

{'_id': 26, 'caption': 'The PPP conversion factor for GDP in 2002 is 0.90497.'}

In [59]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(r['_id'], r['caption'], r.get('ignore')) for r in fix_abbrev)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-05-10 20:18:31.240851: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 20:18:32.327001: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.6s
2023-05-10 20:18:35.625643: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 20:18:35.635466: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-10 20:18:35.677193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warn

In [52]:
res = [g for g in res if len(g['matches']) > 0]
x = lt.err_counter(res)

[('Possible spelling mistake found.', 202145), ('A determiner may be missing.', 51507), ('It appears that a white space is missing.', 21727), ('Don’t put a space before the full stop.', 9195), ('Possible typo: you repeated a whitespace', 6844)]


In [57]:
res[2]

{'question_id': 150,
 'matches': [{'message': 'Possible spelling mistake found.',
   'shortMessage': 'Spelling mistake',
   'replacements': [{'value': 'us'},
    {'value': 'use'},
    {'value': 'USA'},
    {'value': 'USB'},
    {'value': 'USF'},
    {'value': 'USO'},
    {'value': 'USP'},
    {'value': 'USG'},
    {'value': 'USC'},
    {'value': 'USS'},
    {'value': 'US'},
    {'value': 'USD'},
    {'value': 'USE'},
    {'value': 'USI'},
    {'value': 'USJ'},
    {'value': 'USN'},
    {'value': 'USR'},
    {'value': 'usu'}],
   'offset': 24,
   'length': 3,
   'context': {'text': 'The resource balance in us$ in 2007 is equal to the resource balanc...',
    'offset': 24,
    'length': 3},
   'sentence': 'The resource balance in us$ in 2007 is equal to the resource balance in constant us$ in 2008.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'MORFOLOGIK_RULE_EN_US',
    'description': 'Possible spelling mistake',
    'issueType': 'misspelling',
    'category': {'id': 'TYPOS', '